В данном ноутбуке приведены этапы исследования способов ранжирования документов по текстовой релевантности.

При этом максимальный результат получился таким: 0.72044 / 0.74728 (private score / public score).
Вы можете достичь его, последовательно выполняя ячейки разделов: "Подготовка данных", "Новый индекс: с разделением документа на части", "BM25F с расширениями запросов".

## Подготовка данных

In [1]:
import pandas as pd

urls = pd.read_csv('urls.numerate.txt', sep="	", header=None)
urls.columns = ['id', 'url']

queries = pd.read_csv('queries.numerate2.txt', sep="	", header=None)
queries.columns = ['id', 'query']

sample = pd.read_csv('sample.technosphere.ir1.textrelevance.submission.txt', sep=",")

urls_num = urls.shape[0]
print(urls_num)

queries_num = queries.shape[0]
print(queries_num)

doc_ids = {}
for i in range(queries_num):
    doc_ids[i] = list(sample[sample.QueryId == i+1]['DocumentId'])

urls_to_query = {}
for i in range(queries_num):
    urls_list = []
    for j in doc_ids[i]:
        urls_list.append(urls[urls.id == j]['url'][j-1])        
    urls_to_query[queries[queries.id == i+1]['query'][i]] = urls_list

test_url = urls_to_query[queries['query'][7]][15]
print(test_url)

38114
399
nerzon.ru/article/igrat_v_majjnkraft_bez_registracii.html


In [2]:
queries1 = pd.read_csv('queries.numerate1.txt', sep="	", header=None)
queries1.columns = ['id', 'query']

In [4]:
queries['old_query'] = queries1['query']
queries.head()

,id,query,old_query
0,1,мультивиза в израиль какие страны можно посетить,мультивиза в израиль какие страны можно посетить
1,2,надо ли носить компресc гольфы после операции ...,надо ли носить компрес гольфы после операции н...
2,3,жировики на спине можно ли применить пиявки,жировики на спине можно ли применить пиявки
3,4,как прописать просто админку,как прописать просто админку
4,5,кто будет судить суперкубок 2017 між шахтар ди...,хто буде судити суперкубок 2017 між шахтар дин...


In [5]:
from html.parser import HTMLParser
import re
import os
import csv

files = []
for i in range(8):
    directory = 'content/content/'+str(i+1)
    files_list = os.listdir(directory)
    for ind in range(len(files_list)):
        files_list[ind] = directory + '/' + files_list[ind]
    files.append(files_list)

all_files = []
for filenames in files:
    for filename in filenames:
        all_files.append(filename)

"""docs_id = []
n = 0
for file in all_files:
    n += 1
    if n%5000 == 0:
        print(n)
    f = open(file, 'r', encoding='utf-8', errors='ignore').readline()
    url = re.sub('\n', '', f)
    docs_id.append(int(urls[urls.url == url].id))
    
with open('docs_id.txt', 'w', encoding='utf-8', errors='ignore', newline = '') as out:
    writer = csv.writer(out)
    for one_id in docs_id:
        writer.writerow([one_id])
    """

"docs_id = []\nn = 0\nfor file in all_files:\n    n += 1\n    if n%5000 == 0:\n        print(n)\n    f = open(file, 'r', encoding='utf-8', errors='ignore').readline()\n    url = re.sub('\n', '', f)\n    docs_id.append(int(urls[urls.url == url].id))\n    \nwith open('docs_id.txt', 'w', encoding='utf-8', errors='ignore', newline = '') as out:\n    writer = csv.writer(out)\n    for one_id in docs_id:\n        writer.writerow([one_id])\n    "

In [19]:
file = pd.read_csv('docs_id.txt', header=None)
docs_id = list(file[0])

docs_to_query = {}
for i in range(queries_num):
    docs_list = []
    for j in doc_ids[i]:
        docs_list.append(all_files[docs_id.index(j)])        
    docs_to_query[queries[queries.id == i+1]['old_query'][i]] = docs_list

In [7]:
stopwords = pd.read_csv('stop_words_ru.txt', header=None)
stopwords.columns = ['words']

print('как' in list(stopwords.words))

def to_utf8(text):
    if isinstance(text, str): text = text.encode('utf8')
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, str): text = text.decode('utf8')
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

True


In [11]:
urls_count = {url: 0 for url in urls['url']}
urls_heads = {url: url[:url.find('/')] for url in urls['url']}

In [169]:
def calc_links(doc):
    soup = BeautifulSoup(doc, 'html.parser')
    url = re.sub('\n', '', doc)
    ind = url.find('/')
    head_url = url[:ind]
    for link in soup.find_all("a"):
        if link.has_attr('href'):
            cur_link = link['href']
            if (cur_link[:4] != 'http') or (head_url in cur_link):
                continue
            else:
                for url, head_url in urls_heads.items():
                    if head_url in cur_link:
                        urls_count[url] += 1

In [170]:
n = 0
for file in all_files:
    n += 1
    if n%1000 == 0:
        print(n)
    f = open(file, 'r', encoding='utf-8', errors='ignore').read()
    calc_links(f)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


In [8]:
def res_to_ids(res):
    result = []
    for doc in res:
        result.append(docs_id[all_files.index(doc)])
    return result

## Вторая попытка построить индекс (включены английские слова и цифры)

In [112]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = str()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = str()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            word = get_lemmatizer().parse(word)[0].normal_form
            if word not in list(stopwords.words):
                PYMORPHY_CACHE[word_hash] = word
                yield PYMORPHY_CACHE[word_hash]
                
from bs4 import BeautifulSoup

def html2text_bs_visible(raw_html):
    soup = BeautifulSoup(raw_html, 'html.parser')
    [s.extract() for s in soup(['style', 'script', 'head', 'title', 'meta', '[document]'])]
    return soup.get_text()

In [113]:
def html2word(raw_html, to_text=html2text_bs_visible, tokenizer=pymorphy_tokenizer):
    return tokenizer(to_text(raw_html).lower())

In [33]:
def process_files(files):
    file_to_terms = {}
    with open('index1.txt', 'w', encoding='utf-8', errors='ignore') as fout:
        writer = csv.writer(fout)
        for filenames in files:
            print("***")
            for file in filenames:
                PYMORPHY_CACHE = {}
                f = open(file, 'r', encoding='utf-8', errors='ignore').read()
                items = list(html2word(f))
                print(len(items))
                file_to_terms[file] = items
                writer.writerow([file, items])
    return file_to_terms

In [ ]:
k = 300
for query in queries['query'][301:]:
    k += 1
    print('************************************************')
    print(k)
    index1[query] = process_files(query)

In [124]:
def index_one_file(termlist):
    fileIndex = {}
    for index, word in enumerate(termlist):
        if word in fileIndex.keys():
            fileIndex[word].append(index)
        else:
            fileIndex[word] = [index]
    return fileIndex

def make_indices(termlists):
    total = {}
    for filename in termlists.keys():
        items = index_one_file(termlists[filename])
        total[filename] = items
    return total

def fullIndex(index2):
    total_index = {}
    for filename in index2.keys():
        for word in index2[filename].keys():
            if word in total_index.keys():
                if filename in total_index[word].keys():
                    total_index[word][filename].extend(index2[filename][word][:])
                else:
                    total_index[word][filename] = index2[filename][word]
            else:
                total_index[word] = {filename: index2[filename][word]}
    return total_index

In [153]:
with open('new_index14.txt', 'w', encoding='utf-8', errors='ignore', newline='') as fout:
    writer = csv.writer(fout)
    for query in index1.keys():
        for file in index1[query].keys():
            writer.writerow([file, index1[query][file]])

In [171]:
with open('num_of_urls1.txt', 'w', encoding='utf-8', errors='ignore', newline='') as fout:
    writer = csv.writer(fout)
    for url in urls_count.keys():
        writer.writerow([url, urls_count[url]])

In [191]:
index2 = {}
full_index = {}
for query in index1.keys():
    index2[query] = make_indices(index1[query])
    full_index[query] = fullIndex(index2[query])

## BM25 + извлечение признаков из документов
### score: 0.31005 / 0.32990

In [181]:
abbr = pd.read_csv('abbrs.txt', sep=';', header=None)
abbrs = {}
for i in range(len(abbr[0])):
    fname = abbr[0][i]
    sname = abbr[1][i]
    if fname in abbrs.keys():
        abbrs[fname] += ' ' + sname
    else:
        abbrs[fname] = sname

In [183]:
#use first index
def len_of_docs(Index):
    lens = {}
    for file in Index.keys():
        lens[file] = len(Index[file])
    return lens

In [241]:
import requests

def distance(a, b):
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n
    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]

def find_synonyms(word):
    if len(word) == 0:
        return []
    #url = 'http://slova.zkir.ru/dict/'
    url = 'https://synonymonline.ru/'
    url += word[0].title() + '/' + word
    raw_html = requests.get(url)
    raw_html.encoding = 'utf-8'
    soup = BeautifulSoup(raw_html.text)
    syn_list = []
    for item in soup.find_all("span"):
        w = item.string
        if w != 'Меню' and w != None:
            syn_list.append(item.string)
    return syn_list

def find_abbr(word):
    url = 'http://www.sokr.ru/'
    url += word + '/'
    raw_html = requests.get(url)
    raw_html.encoding = 'utf-8'
    soup = BeautifulSoup(raw_html.text)
    abbr_list = []
    for item in soup.find_all("p",  class_="value"):
        if item.string != None:
            abbr_list.append(re.sub('\xa0', ' ', item.string))
    for item in soup.find_all("p",  class_="comment"):
        if item.string != None:
            abbr_list.append(re.sub('\xa0', ' ', item.string))
    return abbr_list

def eng_to_rus_translate(string):
    layout = dict(zip(map(ord, "qwertyuiop[]asdfghjkl;'zxcvbnm,./`"
                           'QWERTYUIOP{}ASDFGHJKL:"ZXCVBNM<>?~'),
                           "йцукенгшщзхъфывапролджэячсмитьбю.ё"
                           'ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ,Ё'))
    return string.translate(layout)

def find_abbr(word):
    url = 'http://www.sokr.ru/'
    url += word + '/'
    raw_html = requests.get(url)
    raw_html.encoding = 'utf-8'
    soup = BeautifulSoup(raw_html.text)
    abbr_list = []
    for item in soup.find_all("p",  class_="value"):
        if item.string != None:
            abbr_list.append(re.sub('\xa0', ' ', item.string))
    for item in soup.find_all("p",  class_="comment"):
        if item.string != None:
            abbr_list.append(re.sub('\xa0', ' ', item.string))
    return abbr_list

def word_query(full_index, word):
    if word in list(stopwords.words):
        return []
    if word in full_index.keys():
        return [filename for filename in full_index[word].keys()]
    else:
        return []

def simple_query(fullIndex, Index1, Index2, string):
    index1_to_query = Index1[string]
    index2_to_query = Index2[string]
    full_index_to_query = fullIndex[string]
    
    new_string = string
    for substr in abbrs.keys():
        if substr in string:
            new_string += substr
    string = re.sub('[ \t\r\n]+', ' ', new_string)
    rank = Rank(string, full_index_to_query, index1_to_query, index2_to_query)
    result = []
    
    for word in string.split():
        word = word.lower()
        word = get_lemmatizer().parse(word)[0].normal_form
        for w in full_index_to_query.keys():
            d = distance(word, w)
            if d <= 2 and d > 0:
                res = word_query(full_index_to_query, w)
                result += res
                for doc in res:
                    if doc in rank.words_to_docs.keys():
                        rank.words_to_docs[doc].append(w)
                    else:
                        rank.words_to_docs[doc] = [w]
                
        synonyms = find_synonyms(word)
        for syn in synonyms:
            for w in syn.split():
                w = get_lemmatizer().parse(w)[0].normal_form
                res = word_query(full_index_to_query, w)
                result += res
                for doc in res:
                    if doc in rank.words_to_docs.keys():
                        rank.words_to_docs[doc].append(w)
                    else:
                        rank.words_to_docs[doc] = [w]
                        
        res = word_query(full_index_to_query, word)
        result += res
        for doc in res:
            if doc in rank.words_to_docs.keys():
                rank.words_to_docs[doc].append(word)
            else:
                rank.words_to_docs[doc] = [word]
        
    if len(rank.words_to_docs.keys()) < 10:
        for word in string.split():
            word = word.lower()
            for w in full_index_to_query.keys():
                d = distance(word, w)
                if d <= 4 and d > 0:
                    res = word_query(full_index_to_query, w)
                    result += res
                    for doc in res:
                        if doc in rank.words_to_docs.keys():
                            rank.words_to_docs[doc].append(w)
                        else:
                            rank.words_to_docs[doc] = [w]
    return rank.search()

In [213]:
import numpy as np

class Rank:
    def __init__(self, query, full_index, index1, index2):
        self.index1 = index1
        self.index2 = index2
        self.frequency = {}
        self.locations = {}
        self.mindistance = {}
        self.words_to_docs = {}
        self.docs_lens = len_of_docs(index1)
        self.query = query
        self.inbound_count = {}
        self.tf_idfs = {}
        
    def normalize_scores(self, scores, smallIsBetter=0):
        vsmall = 0.00001
        if smallIsBetter:
            minscore = min(scores.values())
            return {u: float(minscore)/max(vsmall, l) for (u,l) in scores.items()}
        else:
            maxscore = max(scores.values())
            if maxscore == 0:
                maxscore = vsmall
                return {u: float(c)/maxscore for (u,c) in scores.items()}
        return scores
    
    def frequency_score(self):
        for filename in self.words_to_docs.keys():
            for word in self.words_to_docs[filename]:
                if filename not in self.frequency.keys():
                    self.frequency[filename] = 0
                if word in self.index2[filename].keys():
                    self.frequency[filename] += len(self.index2[filename][word])
        return self.normalize_scores(self.frequency)
                
    def location_score(self):
        for filename in self.words_to_docs.keys():
            words_list = []
            for word in self.words_to_docs[filename]:
                if word in self.index2[filename].keys():
                    words_list += self.index2[filename][word]
            self.locations[filename] = np.median(words_list)/self.docs_lens[filename]
        return self.normalize_scores(self.locations, smallIsBetter=1)
    
    def distance_score(self):
        if len(self.query) <= 1:
            return {file: 1.0 for file in self.words_to_docs.keys()}
        for filename in self.words_to_docs.keys():
            words_list = []
            for word in self.words_to_docs[filename]:
                words_list += self.index2[filename][word]
            words_list.sort()
            if len(words_list) > 1:
                res = sum([abs(words_list[i]-words_list[i-1]) for i in range(1, len(words_list))])
                self.mindistance[filename] = res/len(words_list)
            else:
                self.mindistance[filename] = 1.0
        return self.normalize_scores(self.mindistance, smallIsBetter=1)
    
    def link_score(self):
        inbound_count = {}
        for file in self.words_to_docs.keys():
            f = open(file, 'r', encoding='utf-8', errors='ignore').readline()
            url = re.sub('\n', '', f)
            inbound_count[file] = urls_count[url]
        return self.normalize_scores(inbound_count)
    
    def get_scored_list(self):
        total_scores = {doc: 0 for doc in self.words_to_docs.keys()}
        weight_functions = [(2.0, self.BM25_score()), (1.0, self.frequency_score()), (1.0, self.location_score()), (1.0, self.distance_score()), (1.0, self.link_score())]
        for (weight, scores) in weight_functions:
            for doc in total_scores:
                total_scores[doc] += weight * scores[doc]
        return total_scores
    
    def search(self):
        scores = self.get_scored_list()
        ranked_scores = [(score, doc) for (doc, score) in scores.items()]
        ranked_scores.sort()
        ranked_scores.reverse()
        if len(ranked_scores) >= 10:
            e = 10
        else:
            e = len(ranked_scores)
        return [r[1] for r in ranked_scores[0:e]]
    
    def BM25_score(self):
        k = 2
        b = 0.75
        self.tf_idfs = {doc: 0 for doc in self.words_to_docs.keys()}
        avg_len = np.mean(list(self.docs_lens.values()))
        for word in self.query.split():
            word = word.lower()
            word = get_lemmatizer().parse(word)[0].normal_form
            N = len(index1.keys())
            if word not in full_index.keys():
                continue
            n = len(full_index[word].keys())
            idf = np.log((N - n + 0.5) / (n + 0.5))
            if idf < 0.001:
                idf = 0.001
            for doc in self.words_to_docs.keys():
                all_words = self.index1[doc]
                doc_len = len(all_words)
                tf = all_words.count(word) / doc_len
                if doc in self.tf_idfs.keys():
                    self.tf_idfs[doc] += idf*tf*(k+1)/(tf + k*(1-b+b*doc_len/avg_len))
        return self.normalize_scores(self.tf_idfs)

In [ ]:
import random
import time

result = []
n = 0
with open('new_res1.txt', 'w', encoding='utf-8', errors='ignore', newline='') as out:
    writer = csv.writer(out)
    writer.writerow(['QueryId', 'DocumentId'])
    for q in queries['query']:
        n += 1
        while True:
            try:
                res = simple_query(full_index, index1, index2, q)
                break
            except requests.exceptions.RequestException:
                time.sleep(10)
                
        res = res_to_ids(res)
        print([n, len(res)])
        if len(res) >= 10:
            result += res[:10]
        else:
            result += res
            k = len(res)
            while k < 10:
                rdoc = random.choice(docs_to_query[q])
                rdoc = res_to_ids([rdoc])
                if not (rdoc in res):
                    k += 1
                    result.append(rdoc)
        for i in result[-10:]:
            writer.writerow([n, i])

## Новый индекс: с разделением документа на части

In [240]:
def process_files(query):
    morph = pymorphy2.MorphAnalyzer()
    PATTERN = re.compile(r'[A-Za-zА-Яа-я0-9]+')    
    file_to_terms = {}
    filenames = docs_to_query[query]
    for file in filenames:
        PYMORPHY_CACHE = {}
        f = open(file, 'r', encoding='utf-8', errors='ignore').read()
        lines = list(f)
        raw_html = "".join(lines[1:])
        
        soup = BeautifulSoup(raw_html)
        [s.extract() for s in soup(['script', 'style', 'title', 'meta', '[document]', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
        body = soup.get_text('\n', True).lower()
        body = PATTERN.findall(body)
        body = ' '.join([morph.parse(word)[0].normal_form for word in body])
        
        soup = BeautifulSoup(raw_html)
        title = ' '.join(e.get_text() for e in soup.find_all('title')).lower()
        title = PATTERN.findall(title)
        title = ' '.join([morph.parse(word)[0].normal_form for word in title])
        
        head = ' '.join([e.get_text() for e in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]).lower()
        head = PATTERN.findall(head)
        head = ' '.join([morph.parse(word)[0].normal_form for word in head])
        
        file_to_terms[file] = {}
        file_to_terms[file]['body'] = body
        file_to_terms[file]['title'] = title
        file_to_terms[file]['head'] = head
    with open('{}.txt'.format(query), 'w') as out:
        writer = csv.writer(out)
        for file in filenames:
            writer.writerow([file, file_to_terms[file]])
    return file_to_terms

In [6]:
import numpy as np
import bs4
from bs4 import BeautifulSoup
import re
import os
from tqdm import tqdm
import pymorphy2
from _collections import defaultdict

from multiprocessing.pool import ThreadPool as Pool
from threading import Lock

In [34]:
mutex = Lock()
n_processed = 0
new_index1 = {}

def func_wrapper(q):
    process_files(q) 
    with mutex:
        global n_processed
        n_processed += 1
        print(f"\r{n_processed} objects are processed...", end='', flush=True)
    
with Pool(processes=3) as pool:
    pool.map(func_wrapper, list(queries['old_query']))

399 objects are processed...

## BM25F с расширениями запросов
### score: 0.72044 / 0.74728

In [9]:
def doc_to_dict(query):
    q = query+'.txt'
    new_data = pd.read_csv(q, header=None, engine='python')
    new_data.columns = ['name', 'data']
    res = {}
    for i in range(len(new_data.data)):
        cur_doc = re.sub('[\'\{\},]','', new_data.data[i])
        cur_doc = cur_doc.split(':')
        cur_res = []
        for item in cur_doc[1:]:
            item = ' '.join(item.split()[:-1])
            cur_res.append(item)
        res[new_data.name[i]] = cur_res
    return res

In [10]:
abbrs = pd.read_csv('abbrs.txt', sep = ';', header=None)

In [16]:
csv.field_size_limit(100000000)

131072

In [27]:
from tqdm import tqdm
import pymorphy2
from _collections import defaultdict
import numpy as np

PATTERN = re.compile(r'[A-Za-zА-Яа-я0-9]+')
morph = pymorphy2.MorphAnalyzer()

class BM25F:
    def __init__(self):
        self.total_len = 0
        self.title_len = 0
        self.head_len = 0
        self.body_len = 0
        self.title_idf = defaultdict(float)
        self.head_idf = defaultdict(float)
        self.body_idf = defaultdict(float)
        for query in tqdm(list(queries['old_query'])):
            data = doc_to_dict(query)
            #query_id = queries['id'][queries['old_query'] == query].values[0]
            for doc in docs_to_query[query]:
                self.total_len += 1
                body = PATTERN.findall(data[doc][0].lower())
                self.body_len += len(body)
                self.calc_tf(body, self.body_idf)
                
                title = PATTERN.findall(data[doc][1].lower())
                self.title_len += len(title)
                self.calc_tf(title, self.title_idf)
                
                head = PATTERN.findall(data[doc][2].lower())
                self.head_len += len(head)
                self.calc_tf(head, self.head_idf)
        self.calc_idf(self.body_idf)
        self.body_len /= self.total_len
        self.calc_idf(self.title_idf)
        self.title_len /= self.total_len
        self.calc_idf(self.head_idf)
        self.head_len /= self.total_len
        
    def calc_tf(self, data, idfs_dict):
        included_to_idfs_dict = set()
        for word in data:
            if word not in included_to_idfs_dict:
                idfs_dict[word] += 1
                included_to_idfs_dict.add(word)
                
    def calc_idf(self, idfs_dict):
        total_idf = 0
        negative_idf = []
        for word in idfs_dict:
            idf = np.log((self.total_len - idfs_dict[word] + 0.5)/(idfs_dict[word] + 0.5))
            if idf < 0:
                negative_idf.append(word)
            else:
                idfs_dict[word] = idf
            total_idf += idf
        avg_idf = total_idf / (len(idfs_dict) + len(negative_idf))
        
        for word in negative_idf:
            idfs_dict[word] = 0.1 * avg_idf

    def Rank(self, query):
        old_query = queries['old_query'][queries['query'] == query].values[0]
        doc_score = {}
        for doc in docs_to_query[old_query]:
            doc_score[doc] = self.scoring(query, doc)
        doc_score = sorted(doc_score.items(), key=lambda x: x[1], reverse=True)
        return doc_score

    def scoring(self, query, doc):
        # old_query и query запрос с опечатками и без опечаток
        old_query = queries['old_query'][queries['query'] == query].values[0]
        data = doc_to_dict(old_query)
        body = PATTERN.findall(data[doc][0].lower())
        body_len = len(body)
        body_tf = defaultdict(float)
        for word in body:
            body_tf[word] += 1
        title = PATTERN.findall(data[doc][1].lower())
        title_len = len(title)
        title_tf = defaultdict(float)
        for word in title:
            title_tf[word] += 1
        head = PATTERN.findall(data[doc][2].lower())
        head_len = len(head)
        head_tf = defaultdict(float)
        for word in head:
            head_tf[word] += 1
        
        #добавляем рассширения из файла abbrs.txt, который составлен по сайту sokr.ru
        new_q = query
        for substr1, substr2 in zip(abbrs[0], abbrs[1]):
            if substr1 in query.split():
                new_q += ' ' + substr2
        query = PATTERN.findall(' '.join(morph.parse(word)[0].normal_form for word in new_q.split()))
        k=2.0
        b=0.75
        title_score = 0
        head_score = 0
        body_score = 0
        for word in query:
            #if word in list(stopwords.words):
            #    continue
            body_score += self.body_idf[word] * body_tf[word] * (k + 1) / (body_tf[word] + \
                                                                        k * (1 - b + b * body_len / self.body_len))
            title_score += self.title_idf[word] * title_tf[word] * (k + 1) / (title_tf[word] + \
                                                                        k * (1 - b + b * title_len / self.title_len))
            head_score += self.head_idf[word] * head_tf[word] * (k + 1) / (head_tf[word] + \
                                                                        k * (1 - b + b * head_len / self.head_len))
        #не помню, какие веса дают наилучший score, но начинала с таких:
        return 3.0 * title_score + 1.5 * body_score + 1.0 * head_score

In [29]:
model1 = BM25F()

100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [11:28<00:00,  1.34s/it]


In [233]:
query_res = []
docs_res = []
for query in tqdm(queries['query']):
    res = model1.Rank(query)
    for item in res:
        query_res.append(query)
        docs_res.append(res_to_ids([item[0]])[0])






  0%|                                                                                          | 0/399 [00:00<?, ?it/s]




  0%|▏                                                                               | 1/399 [00:55<6:05:28, 55.10s/it]




  1%|▍                                                                               | 2/399 [01:39<5:42:24, 51.75s/it]




  1%|▌                                                                               | 3/399 [04:11<9:00:03, 81.83s/it]




  1%|▊                                                                               | 4/399 [04:21<6:36:46, 60.27s/it]




  1%|█                                                                               | 5/399 [06:06<8:05:47, 73.98s/it]




  2%|█▏                                                                              | 6/399 [06:20<6:06:29, 55.95s/it]




  2%|█▍                                                                              | 7/399 [06:33<4:40:50, 42.99s/it]




  2

 33%|█████████████████████████▏                                                  | 132/399 [2:12:42<4:43:12, 63.64s/it]




 33%|█████████████████████████▎                                                  | 133/399 [2:12:48<3:25:21, 46.32s/it]




 34%|█████████████████████████▌                                                  | 134/399 [2:13:30<3:18:37, 44.97s/it]




 34%|█████████████████████████▋                                                  | 135/399 [2:16:18<6:00:26, 81.92s/it]




 34%|█████████████████████████▉                                                  | 136/399 [2:16:26<4:22:07, 59.80s/it]




 34%|██████████████████████████                                                  | 137/399 [2:16:33<3:11:35, 43.88s/it]




 35%|██████████████████████████▎                                                 | 138/399 [2:16:37<2:18:18, 31.80s/it]




 35%|██████████████████████████▍                                                 | 139/399 [2:17:55<3:17:44, 45.63s/it]




 35%|███

 66%|██████████████████████████████████████████████████▎                         | 264/399 [4:41:54<2:34:41, 68.75s/it]




 66%|██████████████████████████████████████████████████▍                         | 265/399 [4:42:48<2:23:58, 64.47s/it]




 67%|██████████████████████████████████████████████████▋                         | 266/399 [4:45:46<3:38:18, 98.49s/it]




 67%|██████████████████████████████████████████████████▊                         | 267/399 [4:45:47<2:32:12, 69.18s/it]




 67%|███████████████████████████████████████████████████                         | 268/399 [4:45:50<1:47:34, 49.27s/it]




 67%|███████████████████████████████████████████████████▏                        | 269/399 [4:47:11<2:07:32, 58.86s/it]




 68%|███████████████████████████████████████████████████▍                        | 270/399 [4:47:20<1:34:09, 43.80s/it]




 68%|███████████████████████████████████████████████████▌                        | 271/399 [4:47:56<1:28:42, 41.58s/it]




 68%|███

 99%|█████████████████████████████████████████████████████████████████████████████▍| 396/399 [6:44:42<03:00, 60.04s/it]




 99%|█████████████████████████████████████████████████████████████████████████████▌| 397/399 [6:44:46<01:26, 43.35s/it]




100%|█████████████████████████████████████████████████████████████████████████████▊| 398/399 [6:45:41<00:46, 46.63s/it]




100%|██████████████████████████████████████████████████████████████████████████████| 399/399 [6:46:10<00:00, 41.41s/it]

In [234]:
result = pd.DataFrame({'QueryId': query_res, 'DocumentId': docs_res})

In [236]:
res = []
for query in result_df['QueryId']:
    res.append(queries['id'][queries['query'] == query].values[0])
result['QueryId'] = res

In [238]:
result.to_csv('result5.txt', index=False)

## BM25F -> LDA. Используя результаты result из предыдущего пункта.
### score: 0.69041 / 0.72611

In [48]:
import string
import gensim
import collections
from scipy.spatial import distance

In [94]:
import nltk
nltk.download('stopwords')
stop_words_ru = stopwords.words('russian')
stop_words_en = stopwords.words('english')
stop_words = stop_words_ru + stop_words_en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
def doc_to_dict(query):
    q = query+'.txt'
    new_data = pd.read_csv(q, header=None, engine='python')
    new_data.columns = ['name', 'data']
    docs_ids = res_to_ids(new_data.name.values)
    new_data.name = docs_ids
    res = {}
    for i in range(len(new_data.data)):
        cur_doc = re.sub('[\'\{\},]','', new_data.data[i])
        cur_doc = cur_doc.split(':')
        cur_res = []
        for item in cur_doc[1:]:
            item = ' '.join(item.split()[:-1])
            cur_res.append(item)
        res[new_data.name[i]] = cur_res
    return res

In [162]:
ranged_docs_to_query_head = {}
ranged_docs_to_query = {}
for i in range(399):
    ranged_docs_to_query_head[i+1] = result.DocumentId[result_df['QueryId'] == i+1].values[:10]
    ranged_docs_to_query[i+1] = result.DocumentId[result_df['QueryId'] == i+1].values

In [192]:
def LDA_one_query(texts):
    dictionary = gensim.corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10)
    topicsbook = np.zeros((len(corpus), lda.num_topics))
    for di,doc in enumerate(corpus):
        doc_top = lda[doc]
        for ti,tv in doc_top:
            topicsbook[di,ti] += tv
    weights = topicsbook.sum(0)
    distances = distance.squareform(distance.pdist(topicsbook))
    large = distances.max() + 1
    for i in range(len(distances)):
        distances[i, i] = large
    return (corpus, lda,weights,distances)

In [193]:
def LDA_Rank():
    lda_score = {}
    for query in tqdm(queries['old_query']):
        data = doc_to_dict(query)
        query_text = []
        query_id = queries['id'][queries['old_query'] == query].values[0]
        lda_score[query_id] = {}
        for doc in ranged_docs_to_query_head[query_id]:
            row_str = data[doc][0].split()
            query_text.append([word for word in row_str if word not in stop_words])
        corpus,lda,weights,distances = LDA_one_query(query_text)
        dictionary = gensim.corpora.Dictionary([query.split()])
        query_topic = sorted(lda.get_document_topics(dictionary.doc2bow(query.split())), key=lambda x: x[1], reverse=True)[0][0]
        for doc in ranged_docs_to_query_head[query_id]:
            row_str = data[doc][0].split()
            cur_text = [word for word in row_str if word not in stop_words]
            dictionary = gensim.corpora.Dictionary([cur_text])
            corpus = dictionary.doc2bow(cur_text)
            lda_score[query_id][doc] = 0
            for (topic, val) in lda.get_document_topics(corpus):
                if query_topic == topic:
                    lda_score[query_id][doc] = val
    return lda_score

In [194]:
import warnings
warnings.filterwarnings('ignore')
lda_score = LDA_Rank()






  0%|                                                                                          | 0/399 [00:00<?, ?it/s]




  0%|▏                                                                               | 1/399 [00:28<3:09:22, 28.55s/it]




  1%|▍                                                                               | 2/399 [00:51<2:58:32, 26.98s/it]




  1%|▌                                                                               | 3/399 [01:48<3:56:45, 35.87s/it]




  1%|▊                                                                               | 4/399 [01:57<3:03:28, 27.87s/it]




  1%|█                                                                               | 5/399 [03:00<4:10:55, 38.21s/it]




  2%|█▏                                                                              | 6/399 [03:11<3:17:58, 30.22s/it]




  2%|█▍                                                                              | 7/399 [03:22<2:40:25, 24.55s/it]




  2

 33%|█████████████████████████▏                                                  | 132/399 [1:07:22<2:27:17, 33.10s/it]




 33%|█████████████████████████▎                                                  | 133/399 [1:07:29<1:52:29, 25.37s/it]




 34%|█████████████████████████▌                                                  | 134/399 [1:07:55<1:52:52, 25.56s/it]




 34%|█████████████████████████▋                                                  | 135/399 [1:09:10<2:57:21, 40.31s/it]




 34%|█████████████████████████▉                                                  | 136/399 [1:09:20<2:16:06, 31.05s/it]




 34%|██████████████████████████                                                  | 137/399 [1:09:27<1:44:09, 23.85s/it]




 35%|██████████████████████████▎                                                 | 138/399 [1:09:32<1:19:15, 18.22s/it]




 35%|██████████████████████████▍                                                 | 139/399 [1:10:19<1:56:14, 26.83s/it]




 35%|███

 66%|██████████████████████████████████████████████████▎                         | 264/399 [2:20:55<1:13:19, 32.59s/it]




 66%|██████████████████████████████████████████████████▍                         | 265/399 [2:21:23<1:09:27, 31.10s/it]




 67%|██████████████████████████████████████████████████▋                         | 266/399 [2:22:43<1:41:43, 45.89s/it]




 67%|██████████████████████████████████████████████████▊                         | 267/399 [2:22:45<1:11:58, 32.71s/it]




 67%|████████████████████████████████████████████████████▍                         | 268/399 [2:22:51<53:26, 24.48s/it]




 67%|███████████████████████████████████████████████████▏                        | 269/399 [2:23:28<1:01:29, 28.38s/it]




 68%|████████████████████████████████████████████████████▊                         | 270/399 [2:23:36<47:47, 22.23s/it]




 68%|████████████████████████████████████████████████████▉                         | 271/399 [2:23:55<45:35, 21.37s/it]




 68%|███

 99%|█████████████████████████████████████████████████████████████████████████████▍| 396/399 [3:30:11<01:55, 38.53s/it]




 99%|█████████████████████████████████████████████████████████████████████████████▌| 397/399 [3:30:19<00:58, 29.16s/it]




100%|█████████████████████████████████████████████████████████████████████████████▊| 398/399 [3:31:07<00:34, 34.80s/it]




100%|██████████████████████████████████████████████████████████████████████████████| 399/399 [3:31:32<00:00, 31.89s/it]

In [195]:
QueryId = []
DocumentId = []
for q_id in lda_score.keys():
    sorted_score = sorted(lda_score[q_id].items(), key = lambda x: x[1], reverse=True)
    for doc_id, _ in sorted_score:
        QueryId.append(q_id)
        DocumentId.append(doc_id)

In [196]:
result1 = pd.DataFrame({'QueryId': QueryId, 'DocumentId': DocumentId})

In [197]:
result1.to_csv('result2.txt', index=False)